In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%load_ext tensorboard

import numpy as np
import matplotlib.pyplot as plt
import datetime

from tensorflow import keras
import tensorflow as tf
from tensorflow.python.client import device_lib

import sys
sys.path.append('../')

plt.style.use('./article.mplstyle')

# tf.keras.utils.set_random_seed(1)
# tf.config.experimental.enable_op_determinism()

Tensorflow version and CPU & GPU details.

In [ ]:
print(f'Tensorflow version: {tf.__version__}')
# Check GPU availability.
print(tf.config.list_physical_devices('GPU'))
print('Devices:')
for dev in device_lib.list_local_devices():
    print(dev)

Neural network models.

In [ ]:
output_folder = '../data'
filename = 'synthetic_ppg.npy'
synts, ppgs_raw, ppgs, labels, labels_fixed, noises, model_params = np.load(f'{output_folder}/{filename}', allow_pickle=True)

In [ ]:
def nn_loss(y_true, y_pred):
    y_true /= tf.reduce_sum(y_true)
    y_pred /= tf.reduce_sum(y_pred)
    loss = tf.reduce_sum(
        tf.abs(tf.subtract(tf.cumsum(y_true), tf.cumsum(y_pred))))
    return loss

# Kazemi et al. article (Robust PPG Peak Detection Using Dilated Convolutional Neural Networks).
model_kazemi = tf.keras.Sequential()
model_kazemi.add(keras.layers.Input(shape=(model_params.s_len, 1), name='input'))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=4, activation='elu', dilation_rate=1, padding='same'))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=8, activation='elu', dilation_rate=2, padding='same'))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=8, activation='elu', dilation_rate=4, padding='same'))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=16, activation='elu', dilation_rate=8, padding='same'))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=16, activation='elu', dilation_rate=16, padding='same'))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=32, activation='elu', dilation_rate=32, padding='same',))
model_kazemi.add(keras.layers.Conv1D(kernel_size=3, filters=1, activation='sigmoid', dilation_rate=64, padding='same'))
model_kazemi.compile(optimizer=tf.optimizers.Adam(), loss=nn_loss, metrics=['binary_accuracy'])

# Small model.
model_small = tf.keras.Sequential()
model_small.add(keras.layers.Conv1D(input_shape=(model_params.s_len, 1), kernel_size=5, filters=2, activation='swish', dilation_rate=1, padding='same'))
model_small.add(keras.layers.Normalization())
model_small.add(keras.layers.Conv1D(kernel_size=5, filters=4, activation='swish', dilation_rate=2, padding='same'))
model_small.add(keras.layers.Normalization())
model_small.add(keras.layers.Conv1D(kernel_size=5, filters=8, activation='swish', dilation_rate=4, padding='same'))
model_small.add(keras.layers.Normalization())
model_small.add(keras.layers.Conv1D(kernel_size=5, filters=1, activation='sigmoid', dilation_rate=8, padding='same'))
model_small.compile(optimizer=tf.optimizers.Adam(), loss=nn_loss, metrics=['binary_accuracy'])

# Tiny model.
model_tiny = tf.keras.Sequential()
model_tiny.add(keras.layers.Conv1D(input_shape=(model_params.s_len, 1), kernel_size=5, filters=2, activation='swish', dilation_rate=2, padding='same'))
model_tiny.add(keras.layers.Normalization())
model_tiny.add(keras.layers.Conv1D(kernel_size=5, filters=1, activation='sigmoid', dilation_rate=4, padding='same'))
model_tiny.compile(optimizer=tf.optimizers.Adam(), loss=nn_loss, metrics=['binary_accuracy'])

model_output_dir = 'models'

model_names = ['tiny', 'small', 'kazemi']
models = [model_tiny, model_small, model_kazemi]

In [ ]:
batch_size = 256
steps_per_epoch = None
epochs = 20
nn_labels = labels_fixed

# Split the dataset into training (80%), validation (10%) and testing (10%).
ds_len = len(ppgs)
x_train = ppgs[:int(0.8 * ds_len)]
y_train = nn_labels[:int(0.8 * ds_len)]
x_val = ppgs[int(0.8 * ds_len):int(0.9 * ds_len)]
y_val = nn_labels[int(0.8 * ds_len):int(0.9 * ds_len)]
x_test = ppgs[int(0.9 * ds_len):]
y_test = nn_labels[int(0.9 * ds_len):]
assert len(x_train) + len(x_val) + len (x_test) == ds_len, 'Training, validation and test dataset lengths do not match that of the whole dataset.'

for model_name, model in zip(model_names, models):
    # Train the model.
    csv_logger = keras.callbacks.CSVLogger(f'../{model_output_dir}/{model_name}.log')
    log_dir = "../tensorboard_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    history = model.fit(x_train, y_train, batch_size=batch_size, 
        steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=(x_val, y_val), 
        callbacks=[csv_logger, tensorboard_callback])

    # Print model details.
    model.summary()

    # Run the test set.
    test_res = model.evaluate(x_test, y_test)
    print('Test loss, test accuracy:', test_res)

    # Plot history.
    fig, axes = plt.subplots(1, 2, figsize=(8, 5))
    axes[0].set_title('Loss')
    axes[0].plot(history.epoch, history.history['loss'], label='Training loss')
    axes[0].plot(history.epoch, history.history['val_loss'],
                    label='Validation loss')
    axes[1].set_title('Accuracy')
    axes[1].plot(history.epoch, history.history['binary_accuracy'],
                    label='Training accuracy')
    axes[1].plot(history.epoch, history.history['val_binary_accuracy'],
                    label='Validation accuracy')
    axes[0].legend()
    axes[1].legend()

    # Save the model and convert it into a TF Lite model.
    run_model = tf.function(lambda x: model(x))
    # This is important, let's fix the input size.
    concrete_func = run_model.get_concrete_function(
        tf.TensorSpec([1, model_params.s_len, 1], model.inputs[0].dtype))

    # Save the model.
    model.save(f'../{model_output_dir}/{model_name}', save_format="tf", signatures=concrete_func)

    # Convert to TF Lite model.
    converter = tf.lite.TFLiteConverter.from_saved_model(f'../{model_output_dir}/{model_name}')
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Analyze the model.
    tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)
    # Save the model.
    open(f'../{model_output_dir}/{model_name}.tflite', 'wb').write(tflite_model)

In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, model_params.s_len, 1], model.inputs[0].dtype))

# Save the model.
model.save(f'../{model_output_dir}/{model_name}', save_format="tf", signatures=concrete_func)

# Convert to TF Lite model.
converter = tf.lite.TFLiteConverter.from_saved_model(f'../{model_output_dir}/{model_name}')
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Analyze the model.
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)
# Save the model.
open(f'../{model_output_dir}/{model_name}.tflite', 'wb').write(tflite_model)

Navigate to the models dir and run the following command in e.g. Git Bash to convert the model to C array:
```console
xxd -i model.tflite > model.cc
```